In [3]:
import gym      
import snake_gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.02
gamma         = 0.9
buffer_limit  = 30000
batch_size    = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, data):
        self.buffer.append(data)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(100, 150)
        self.fc2 = nn.Linear(150, 75)
        self.fc3 = nn.Linear(75, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        obs = torch.reshape(obs, (-1,))
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 3)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(20):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        s = torch.reshape(s, (batch_size, 10*10))
        s_prime = torch.reshape(s_prime, (batch_size, 10*10))
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('Snake-v0')
    env.energy_consum = True
    
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    render = False

    for n_epi in range(10000):
        env.seed(1)
        epsilon = max(0.01, 0.1 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False
        #print('New Episode!')
        #print()
        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            #print(a)
            s_prime, r, done, info = env.step(a)
            #print(r, end='')
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r,s_prime, done_mask))
            s = s_prime

            score += r
            
            if render:
                env.render()
                
            if done:
                break
            
        if score/print_interval > 50:
            render = True
            
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : -1.1, n_buffer : 47, eps : 9.9%
n_episode :40, score : -1.0, n_buffer : 88, eps : 9.8%
n_episode :60, score : -1.0, n_buffer : 130, eps : 9.7%
n_episode :80, score : -1.0, n_buffer : 172, eps : 9.6%
n_episode :100, score : -1.0, n_buffer : 215, eps : 9.5%
n_episode :120, score : -1.0, n_buffer : 260, eps : 9.4%
n_episode :140, score : -1.0, n_buffer : 302, eps : 9.3%
n_episode :160, score : -1.0, n_buffer : 343, eps : 9.2%
n_episode :180, score : -1.0, n_buffer : 388, eps : 9.1%
n_episode :200, score : -1.0, n_buffer : 431, eps : 9.0%
n_episode :220, score : -1.0, n_buffer : 472, eps : 8.9%
n_episode :240, score : -1.0, n_buffer : 514, eps : 8.8%
n_episode :260, score : -1.0, n_buffer : 557, eps : 8.7%
n_episode :280, score : -1.0, n_buffer : 604, eps : 8.6%
n_episode :300, score : -1.0, n_buffer : 645, eps : 8.5%
n_episode :320, score : -1.0, n_buffer : 693, eps : 8.4%
n_episode :340, score : -1.0, n_buffer : 735, eps : 8.3%
n_episode :360, score : -1.0, n_buffe

KeyboardInterrupt: 